In [11]:
from moabb.datasets import BNCI2014001, Cho2017, PhysionetMI,Zhou2016
from moabb.paradigms import MotorImagery


from mne  import concatenate_epochs

import os
import numpy as np
import itertools
import pandas as pd
import pickle


path = '/users/local/simpleconv_datasets/'



|           | BNCI | CHO2017 | Physionet| Weibo2014 | Zhou2016 | Shin2017 |
| --------  | ---- | -----   | -------- | --------- | -------- | -------- |
| BNCI      |22(+2)|   22    |   22     |    22     |     9    |    2     | 
| CHO2017   |  22  |  64(+5) |   62     |    58     |    14    |   13     |  
| Physionet |  22  |  62    |   64     |   58      |   14     |   12     | 
| Weibo2014 |  22  |  58     |  65     |   60      |   14     |   12     | 
| Zhou2016  |  9   |   14    |   14     |  14       |  14(+2)  |   1      |   
| Shin2017  |  2   |   13    |   12     |  12       |   1      |  30(+3)  | 

In [12]:
def load_data(dset,num_classes,selected_chans = None,subjects=None,sfreq = 160,fmin=2,fmax=40,tmin=0,tmax=3):
    print(num_classes)
    ds_src = dset()
    src_prgm = MotorImagery(n_classes=num_classes, channels=selected_chans, resample=sfreq, fmin=fmin, fmax=fmax, tmin=tmin,tmax=tmax)
    if not subjects:
        epoch_X_src, label_src, m_src = src_prgm.get_data(dataset=ds_src, return_epochs=True)
    else : 
        epoch_X_src, label_src, m_src = src_prgm.get_data(dataset=ds_src,subjects=subjects, return_epochs=True)
    return epoch_X_src,label_src,m_src

In [13]:
import torch
from scipy.linalg import sqrtm, inv,pinv
from sklearn.preprocessing import LabelEncoder
import gc
import os

def compute_r_op(X):
    r = torch.einsum('bet, tab -> bea',X,X.T).mean(0)
    r_op = torch.from_numpy(inv(sqrtm(r)))
    return r_op

def prepro_data_session(dset_name_path):
    with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "rb") as input_file:
        epoch_X = pickle.load(input_file)    
        
    m_src = pd.read_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'))
    label_src = np.load(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'))
    X_list = []
    X_list_raw = []
    Y_list = []
    #X_list_raw = []
    encoder = LabelEncoder()
    Y = torch.from_numpy(encoder.fit_transform(label_src)).long()
    X = torch.from_numpy(epoch_X.get_data()).float()
    del epoch_X,label_src
    
    for s_id in m_src.subject.unique():
        X_list_sub = []
        Y_list_sub = []
        X_list_raw_sub = []
        m_src_i = m_src[m_src['subject']==s_id]
        for session in m_src_i.session.unique():
            idx = m_src_i[m_src_i['session']==session].index.tolist()
            X_s = X[idx]
            Y_s = Y[idx]
            X_list_raw_sub.append(X_s)
            sqrt_R_s = compute_r_op(X_s).float()
            X_list_sub.append(torch.einsum("fe,bet->bft",sqrt_R_s, X_s))
            Y_list_sub.append(Y_s)
            del X_s,Y_s,sqrt_R_s
            gc.collect()
        X_list.append(X_list_sub)
        X_list_raw.append(X_list_raw_sub)
        Y_list.append(Y_list_sub)
        
    torch.save(X_list_raw,os.path.join(path,dset_name_path+'/X_s.pt'))
    torch.save(X_list, os.path.join(path,dset_name_path+'/X_EA_s.pt'))
    torch.save(Y_list, os.path.join(path,dset_name_path+'/Y_s.pt'))
    #return X_list,X_list_raw,Y_list
        
def prepro_data(dset_name_path):
    with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "rb") as input_file:
        epoch_X = pickle.load(input_file)    
        
    m_src = pd.read_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'))
    label_src = np.load(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'))
    X_list = []
    Y_list = []
    X_list_raw = []
    encoder = LabelEncoder()
    Y = torch.from_numpy(encoder.fit_transform(label_src)).long()
    X = torch.from_numpy(epoch_X.get_data()).float()
    del epoch_X,label_src
    for s_id in m_src.subject.unique():
        idx = m_src[m_src['subject']==s_id].index.tolist()
        X_s = X[idx]
        Y_s = Y[idx]
        sqrt_R_s = compute_r_op(X_s).float()
        X_list.append(torch.einsum("fe,bet->bft",sqrt_R_s, X_s))
        X_list_raw.append(X_s)
        Y_list.append(Y_s)
        del X_s,Y_s,sqrt_R_s
        gc.collect()
        
    torch.save(X_list_raw,os.path.join(path,dset_name_path+'/X.pt'))
    torch.save(X_list, os.path.join(path,dset_name_path+'/X_EA.pt'))
    torch.save(Y_list, os.path.join(path,dset_name_path+'/Y.pt'))

## BNCI

3 derniers channels = EOG
offset de 0.5 = 0.5 * 250 = 125

In [14]:
chans = ['Fz','FC3','FC1','FCz','FC2','FC4','C5','C3','C1','Cz','C2','C4','C6','CP3','CP1','CPz','CP2','CP4','P1','Pz','P2','POz']

n_c = 4
epoch_X_b,label_src_b,m_src_b =load_data(BNCI2014001,n_c,selected_chans = chans,
                                        sfreq = 250,fmin=0,fmax=124.99,tmin=0,tmax=4) #250

dset_name_path = 'BNCI'

with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "wb") as output_file:
    pickle.dump(epoch_X_b, output_file)
    
m_src_b.to_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'),index=False)
np.save(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'), label_src_b)

BNCI2014001 has been renamed to BNCI2014_001. BNCI2014001 will be removed in version 1.1.
The dataset class name 'BNCI2014001' must be an abbreviation of its code 'BNCI2014-001'. See moabb.datasets.base.is_abbrev for more information.
Choosing from all possible events


4


/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'lef

Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns


/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/usr/local/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 2 – 6 s (baseline off), ~8.1 MB, data loaded,
 'lef

Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding metadata with 3 columns
Adding m

/usr/local/lib/python3.10/site-packages/moabb/paradigms/base.py:350: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)


Adding metadata with 3 columns
5184 matching events found
No baseline correction applied


FileNotFoundError: [Errno 2] No such file or directory: '/users/local/simpleconv_datasets/BNCI/BNCI_epoch.pkl'

In [ ]:
prepro_data(dset_name_path)
prepro_data_session(dset_name_path)

## Zhou

In [ ]:
#chans = ['Fz','FC3','FC1','FCz','FC2','FC4','C5','C3','C1','Cz','C2','C4','C6','CP3','CP1','CPz','CP2','CP4','P1','Pz','P2','POz',
#             'EOG1','EOG2','EOG3']

Zhou_chan  = ['Fp1','Fp2','FC3','FCz','FC4','C3','Cz','C4','CP3','CPz','CP4','O1','Oz','O2']


n_c = 3
epoch_X_z,label_src_z,m_src_z =load_data(Zhou2016,n_c,selected_chans = Zhou_chan,
                                        sfreq = 250,fmin=0.5,fmax=124.99,tmin=0,tmax=5) #250



In [ ]:
dset_name_path = 'Zhou'

with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "wb") as output_file:
    pickle.dump(epoch_X_z, output_file)
    
m_src_z.to_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'),index=False)
np.save(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'), label_src_z)

In [ ]:
prepro_data(dset_name_path)
prepro_data_session(dset_name_path)

## Physionet

Subjects exclusions References : 
- Karel Roots, Yar Muhammad, and Naveed Muhammad. Fusion convolutional neural network for cross-subject eeg motor imagery classification. Computers, 9(3):72, 2020.
- Jason Sleight, Preeti Pillai, and Shiwali Mohan. Classification of executed and imagined motor movement eeg signals. Ann Arbor: University of Michigan, 110, 2009.

In [ ]:

Physionet_chan = ['FC5','FC3','FC1','FCz','FC2','FC4','FC6','C5','C3','C1','Cz','C2','C4','C6','CP5','CP3','CP1','CPz','CP2',
                  'CP4','CP6','Fp1','Fpz','Fp2','AF7','AF3','AFz','AF4','AF8','F7','F5','F3','F1','Fz','F2','F4','F6','F8',
                  'FT7','FT8','T7','T8','T9','T10','TP7','TP8','P7','P5','P3','P1','Pz','P2','P4','P6','P8','PO7','PO3',
                  'POz','PO4','PO8','O1','Oz','O2','Iz']

n_c = 4
subjects_physio = [x for x in list(np.arange(1,110)) if x not in [38,88,89,92,100,104]]

epoch_X_p,label_src_p,m_src_p =load_data(PhysionetMI,n_c,selected_chans = Physionet_chan,
                                        sfreq = 160,fmin=0,fmax=79.99,tmin=0,tmax=3.5,
                                        subjects = subjects_physio) # 160
                                         #subjects=subjects_physio)

dset_name_path = 'Physionet'

with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "wb") as output_file:
    pickle.dump(epoch_X_p, output_file)
    
m_src_p.to_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'),index=False)
np.save(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'), label_src_p)

In [ ]:
prepro_data(dset_name_path)

# Cho
Subjects exclusions References : 
- Vinay Jayaram and Alexandre Barachant. Moabb: trustworthy al- gorithm benchmarking for bcis. Journal of neural engineering, 15(6):066011, 2018.

In [ ]:
Cho2017_chan = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5','FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1',
                'P3','P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2','AF8','AF4','AFz','Fz','F2','F4','F6'
                ,'F8','FT8','FC6','FC4','FC2','FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4','P6','P8','P10','PO8','PO4','O2']
n_c = 2
subjects_cho = [x for x in list(np.arange(1,53)) if x not in [32, 46, 49]]
epoch_X_c,label_src_c,m_src_c =load_data(Cho2017,n_c,selected_chans = Cho2017_chan,
                                         sfreq = 512,fmin=0.5,fmax=255.99,tmin=0,tmax=3,
                                         subjects = subjects_cho) # 512

dset_name_path = 'Cho'

with open(os.path.join(path,dset_name_path+'/'+dset_name_path+'_epoch.pkl'), "wb") as output_file:
    pickle.dump(epoch_X_c, output_file)
    
m_src_c.to_csv(os.path.join(path,dset_name_path+'/'+dset_name_path+'_m.csv'),index=False)
np.save(os.path.join(path,dset_name_path+'/'+dset_name_path+'_label.npy'), label_src_c)

In [ ]:
prepro_data(dset_name_path)